In [ ]:
import re
import pandas as pd
import numpy as np
from PyPDF2 import PdfReader
import json
import os
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import DirectoryLoader, PyPDFLoader
from tqdm.notebook import tqdm
from langchain.schema import Document

In [5]:
uu_file_path = "../data/UU Nomor 13 Tahun 2003.pdf"
print(f"File UU No. 13 Tahun 2003 sudah ada di {uu_file_path}")

File UU No. 13 Tahun 2003 sudah ada di ../data/UU Nomor 13 Tahun 2003.pdf


In [16]:
loader = PyPDFLoader(uu_file_path)
documents = loader.load()
print(f"Jumlah dokumen yang dimuat: {len(documents)}")

Jumlah dokumen yang dimuat: 128


In [2]:
try:
    with open('UU_Nomor_13_Tahun_2003_Ketenagakerjaan.txt', 'r', encoding='utf-8') as f:
        uu_text = f.read()
    print("Berhasil membaca file teks yang sudah ada.")
except:
    print("File teks tidak ditemukan. Menggunakan teks yang ada di variabel.")

File teks tidak ditemukan. Menggunakan teks yang ada di variabel.


Teknik Chunking dengan Mempertahankan Struktur Bab, Pasal, dan Ayat


In [ ]:
def extract_structure(text):
    """
    Mengekstrak struktur UU menjadi dict dengan struktur bab, pasal, dan ayat
    """
    # Pola untuk mendeteksi BAB
    bab_pattern = r'BAB\s+([IVXLCDM]+)\s*\n(.*?)\n'
    
    # Pola untuk mendeteksi Pasal
    pasal_pattern = r'Pasal\s+(\d+)\s*\n'
    
    # Pola untuk mendeteksi Ayat
    ayat_pattern = r'\((\d+)\)\s+'
    
    structured_law = {}
    current_bab = None
    current_pasal = None
    
    # Split text berdasarkan baris
    lines = text.split('\n')
    i = 0
    
    while i < len(lines):
        line = lines[i]
        
        # Cek apakah ini adalah BAB baru
        bab_match = re.search(r'BAB\s+([IVXLCDM]+)', line)
        if bab_match:
            # Jika ada baris selanjutnya yang merupakan judul BAB
            if i + 1 < len(lines) and not re.search(r'BAB\s+([IVXLCDM]+)', lines[i+1]):
                current_bab = f"BAB {bab_match.group(1)}: {lines[i+1].strip()}"
                structured_law[current_bab] = {}
                i += 2
                continue
            else:
                current_bab = f"BAB {bab_match.group(1)}"
                structured_law[current_bab] = {}
        
        # Cek apakah ini adalah Pasal baru
        pasal_match = re.search(r'Pasal\s+(\d+)', line)
        if pasal_match and current_bab is not None:
            current_pasal = f"Pasal {pasal_match.group(1)}"
            structured_law[current_bab][current_pasal] = {"text": "", "ayat": {}}
            
            # Ambil isi pasal sampai menemukan pasal baru atau bab baru
            pasal_content = []
            j = i + 1
            while j < len(lines):
                next_line = lines[j]
                if re.search(r'Pasal\s+\d+', next_line) or re.search(r'BAB\s+[IVXLCDM]+', next_line):
                    break
                    
                # Deteksi Ayat
                ayat_match = re.search(r'^\s*\((\d+)\)\s+(.+)', next_line)
                if ayat_match:
                    ayat_num = ayat_match.group(1)
                    ayat_text = ayat_match.group(2)
                    
                    # Tambahkan ayat ke struktur
                    if current_pasal in structured_law[current_bab]:
                        structured_law[current_bab][current_pasal]["ayat"][f"Ayat {ayat_num}"] = ayat_text
                else:
                    pasal_content.append(next_line)
                
                j += 1
            
            # Gabungkan isi pasal
            if pasal_content:
                structured_law[current_bab][current_pasal]["text"] = "\n".join(pasal_content).strip()
                
            i = j - 1  # Sesuaikan iterator
        
        i += 1
    
    return structured_law


Fungsi untuk menghasilkan chunks dengan metadata struktur


In [13]:
def create_chunks_with_structure(text, chunk_size=500, chunk_overlap=50):
    
    # Membuat chunks dengan struktur Bab, Pasal, dan Ayat sebagai metadata
    # Ekstrak struktur terlebih dahulu
    structured_law = extract_structure(text)
    
    # Buat text splitter
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=chunk_size, 
        chunk_overlap=chunk_overlap,
        separators=["\n\n", "\n", " ", ""]
    )
    
    # Proses chunks dengan metadata struktur
    chunks = []
    
    for bab, pasal_dict in structured_law.items():
        for pasal, content in pasal_dict.items():
            # Proses teks pasal
            pasal_text = content["text"]
            if pasal_text:
                pasal_chunks = text_splitter.create_documents(
                    [pasal_text], 
                    metadatas=[{
                        "source": "UU No 13 Tahun 2003",
                        "bab": bab,
                        "pasal": pasal,
                        "type": "pasal_text"
                    }]
                )
                chunks.extend(pasal_chunks)
            
            # Proses ayat-ayat
            for ayat, ayat_text in content["ayat"].items():
                ayat_chunks = text_splitter.create_documents(
                    [ayat_text], 
                    metadatas=[{
                        "source": "UU No 13 Tahun 2003",
                        "bab": bab,
                        "pasal": pasal,
                        "ayat": ayat,
                        "type": "ayat"
                    }]
                )
                chunks.extend(ayat_chunks)
    
    return chunks


Alternatif pendekatan chunking: Mempertahankan konteks lengkap


In [14]:
def create_context_aware_chunks(text, chunk_size=1000, chunk_overlap=200):
    """
    Membuat chunks dengan konteks hierarchy BAB > Bagian > Pasal > Ayat
    """
    # Pola-pola untuk deteksi struktur
    bab_pattern = r'BAB\s+([IVXLCDM]+)\s+([^\n]+)'
    bagian_pattern = r'Bagian\s+([A-Za-z0-9]+)\s+([^\n]+)'
    pasal_pattern = r'Pasal\s+(\d+)(\s+[^\n]+)?'
    ayat_pattern = r'\((\d+)\)\s+'
    
    # Split text berdasarkan baris
    lines = text.split('\n')
    
    chunks = []
    current_bab = None
    current_bagian = None
    current_pasal = None
    current_chunk_text = ""
    current_chunk_metadata = {}
    
    for line in lines:
        # Deteksi BAB baru
        bab_match = re.search(bab_pattern, line)
        if bab_match:
            # Simpan chunk sebelumnya jika ada
            if current_chunk_text and len(current_chunk_text) > 50:  # minimum 50 char untuk chunk
                chunks.append({
                    "text": current_chunk_text.strip(),
                    "metadata": current_chunk_metadata.copy()
                })
            
            # Mulai chunk baru untuk BAB
            current_bab = f"BAB {bab_match.group(1)}: {bab_match.group(2).strip()}"
            current_bagian = None  # Reset bagian
            current_pasal = None   # Reset pasal
            current_chunk_text = f"{current_bab}\n"
            current_chunk_metadata = {
                "source": "UU No 13 Tahun 2003",
                "bab": current_bab,
                "bagian": None,
                "pasal": None,
                "ayat": None,
                "type": "bab_title"
            }
            continue
        
        # Deteksi Bagian baru
        bagian_match = re.search(bagian_pattern, line)
        if bagian_match:
            # Simpan chunk sebelumnya jika ada
            if current_chunk_text and len(current_chunk_text) > 50:
                chunks.append({
                    "text": current_chunk_text.strip(),
                    "metadata": current_chunk_metadata.copy()
                })
            
            # Mulai chunk baru untuk Bagian
            current_bagian = f"Bagian {bagian_match.group(1)}: {bagian_match.group(2).strip()}"
            current_pasal = None  # Reset pasal
            current_chunk_text = f"{current_bagian}\n"
            current_chunk_metadata = {
                "source": "UU No 13 Tahun 2003",
                "bab": current_bab,
                "bagian": current_bagian,
                "pasal": None,
                "ayat": None,
                "type": "bagian_title"
            }
            continue
        
        # Deteksi Pasal baru
        pasal_match = re.search(pasal_pattern, line)
        if pasal_match:
            # Simpan chunk sebelumnya jika ada
            if current_chunk_text and len(current_chunk_text) > 50:
                chunks.append({
                    "text": current_chunk_text.strip(),
                    "metadata": current_chunk_metadata.copy()
                })
            
            # Mulai chunk baru untuk Pasal
            current_pasal = f"Pasal {pasal_match.group(1)}"
            current_chunk_text = f"{current_pasal}\n"
            current_chunk_metadata = {
                "source": "UU No 13 Tahun 2003",
                "bab": current_bab,
                "bagian": current_bagian,
                "pasal": current_pasal,
                "ayat": None,
                "type": "pasal_header"
            }
            continue
        
        # Deteksi Ayat
        ayat_match = re.search(ayat_pattern, line)
        if ayat_match and current_pasal:
            # Tambahkan ayat ke chunk saat ini
            ayat_num = ayat_match.group(1)
            ayat_content = line[ayat_match.end():]
            
            # Simpan chunk sebelumnya jika teksnya sudah panjang
            if len(current_chunk_text) > chunk_size:
                chunks.append({
                    "text": current_chunk_text.strip(),
                    "metadata": current_chunk_metadata.copy()
                })
                # Mulai chunk baru tapi pertahankan konteks
                current_chunk_text = f"{current_bab}\n{current_pasal}\n"
            
            current_chunk_text += f"({ayat_num}) {ayat_content}\n"
            current_chunk_metadata["ayat"] = f"Ayat {ayat_num}"
            current_chunk_metadata["type"] = "pasal_with_ayat"
            continue
        
        # Tambahkan line ke chunk saat ini
        if line.strip():
            current_chunk_text += line + "\n"
            
            # Cek jika chunk saat ini melebihi ukuran
            if len(current_chunk_text) > chunk_size + chunk_overlap:
                chunks.append({
                    "text": current_chunk_text[:chunk_size].strip(),
                    "metadata": current_chunk_metadata.copy()
                })
                # Simpan bagian yang overlap untuk chunk berikutnya
                current_chunk_text = current_chunk_text[chunk_size-chunk_overlap:]
    
    # Tambahkan chunk terakhir jika ada
    if current_chunk_text and len(current_chunk_text) > 50:
        chunks.append({
            "text": current_chunk_text.strip(),
            "metadata": current_chunk_metadata.copy()
        })
    
    return chunks

Jalankan proses chunking

In [17]:
# Metode 1: Chunks dengan struktur yang terekstrak
print("Melakukan chunking dengan struktur...")
# Gabungkan teks dari semua dokumen menjadi satu string
documents_text = "\n".join([doc.page_content for doc in documents])

# Lakukan chunking dengan struktur
chunks_structured = create_chunks_with_structure(documents_text, chunk_size=1000, chunk_overlap=100)
print(f"Dihasilkan {len(chunks_structured)} chunks dengan struktur.")

Melakukan chunking dengan struktur...
Dihasilkan 671 chunks dengan struktur.


In [19]:
# Metode 2: Chunks dengan kesadaran konteks
print("Melakukan chunking dengan kesadaran konteks...")
chunks_context_aware = create_context_aware_chunks(documents_text, chunk_size=1000, chunk_overlap=200)
print(f"Dihasilkan {len(chunks_context_aware)} chunks dengan konteks.")

Melakukan chunking dengan kesadaran konteks...
Dihasilkan 376 chunks dengan konteks.


In [20]:
# Konversi chunks struktur ke dataframe
def chunks_to_dataframe(chunks, method_name):
    if method_name == "structured":
        df = pd.DataFrame([{
            "text": chunk.page_content,
            "bab": chunk.metadata.get("bab", ""),
            "pasal": chunk.metadata.get("pasal", ""),
            "ayat": chunk.metadata.get("ayat", ""),
            "type": chunk.metadata.get("type", ""),
            "source": chunk.metadata.get("source", ""),
        } for chunk in chunks])
    else:
        df = pd.DataFrame([{
            "text": chunk["text"],
            "bab": chunk["metadata"].get("bab", ""),
            "bagian": chunk["metadata"].get("bagian", ""),
            "pasal": chunk["metadata"].get("pasal", ""),
            "ayat": chunk["metadata"].get("ayat", ""),
            "type": chunk["metadata"].get("type", ""),
            "source": chunk["metadata"].get("source", ""),
        } for chunk in chunks])
    
    return df

df_structured = chunks_to_dataframe(chunks_structured, "structured")
df_context_aware = chunks_to_dataframe(chunks_context_aware, "context_aware")

# Simpan dataframe ke CSV untuk digunakan dalam sistem RAG
df_structured.to_csv("uu_ketenagakerjaan_chunks_structured.csv", index=False)
df_context_aware.to_csv("uu_ketenagakerjaan_chunks_context_aware.csv", index=False)

print("Chunking selesai dan data disimpan ke CSV.")

Chunking selesai dan data disimpan ke CSV.


In [21]:
# =======================================
# Analisis distribusi chunk
# =======================================

print("\nAnalisis distribusi chunk structured:")
print(f"Jumlah chunks: {len(df_structured)}")
print(f"Rata-rata panjang chunk: {df_structured['text'].str.len().mean():.2f} karakter")
print(f"Min panjang chunk: {df_structured['text'].str.len().min()} karakter")
print(f"Max panjang chunk: {df_structured['text'].str.len().max()} karakter")

print("\nAnalisis distribusi chunk context aware:")
print(f"Jumlah chunks: {len(df_context_aware)}")
print(f"Rata-rata panjang chunk: {df_context_aware['text'].str.len().mean():.2f} karakter")
print(f"Min panjang chunk: {df_context_aware['text'].str.len().min()} karakter")
print(f"Max panjang chunk: {df_context_aware['text'].str.len().max()} karakter")

print("\nAnalisis referensi struktur:")
print(f"Jumlah BAB unik: {df_context_aware['bab'].nunique()}")
print(f"Jumlah Pasal unik: {df_context_aware['pasal'].nunique()}")


Analisis distribusi chunk structured:
Jumlah chunks: 671
Rata-rata panjang chunk: 168.44 karakter
Min panjang chunk: 11 karakter
Max panjang chunk: 1000 karakter

Analisis distribusi chunk context aware:
Jumlah chunks: 376
Rata-rata panjang chunk: 444.72 karakter
Min panjang chunk: 55 karakter
Max panjang chunk: 1188 karakter

Analisis referensi struktur:
Jumlah BAB unik: 8
Jumlah Pasal unik: 183


In [22]:
print("\nContoh chunks dari metode structured:")
print(df_structured.head(2)[["text", "bab", "pasal", "ayat"]].to_string())

print("\nContoh chunks dari metode context aware:")
print(df_context_aware.head(2)[["text", "bab", "pasal", "ayat"]].to_string())

# Fungsi untuk mencari chunk berdasarkan pasal
def search_chunks_by_pasal(df, pasal_number):
    return df[df['pasal'].str.contains(f"Pasal {pasal_number}", na=False)]

# Contoh pencarian pasal
pasal_to_search = 10  # Ganti dengan nomor pasal yang ingin dicari
print(f"\nContoh pencarian Pasal {pasal_to_search}:")
pasal_chunks = search_chunks_by_pasal(df_context_aware, pasal_to_search)
if not pasal_chunks.empty:
    print(pasal_chunks[["text", "bab", "pasal", "ayat"]].to_string())
else:
    print(f"Pasal {pasal_to_search} tidak ditemukan.")


Contoh chunks dari metode structured:
                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                 